# Homework 6

Student: Ante Malenica

## Part I

1. Go get data from kaggle.com and do a ***(Univariate) Bayesian Logistic Regression*** analysis

2. Adjust the code below to specify that the outcomes have a Bernoulli distribution and use a ***logit*** or ***probit link function*** (or $\Pr(z\leq 0)$ for latent $z$ ) to correctly parameterize the predicted values of the observed outcomes

```python
import pymc as pm; import numpy as np
n,p=100,10; X,y=np.zeros((n,p)),np.ones((n,1))
# Replace this made up data with your data set from kaggle...
with pm.Model() as MLR:
    betas = pm.MvNormal('betas', mu=np.zeros((p,1)), cov=np.eye(p), shape=(p,1))
    sigma = pm.TruncatedNormal('sigma', mu=1, sigma=1, lower=0) # half normal
    y = pm.Normal('y', mu=pm.math.dot(X, betas), sigma=sigma, observed=y)

with MLR:
    idata = pm.sample()
```    

3. Choose ***prior*** that are sensible for your specification

4. [Optional] Assess the performance of the MCMC and any issues or warnings in the [standard manner](https://www.pymc.io/projects/docs/en/stable/learn/core_notebooks/pymc_overview.html#pymc-overview)

4. [Optional] Go get data from kaggle.com and do a ***Multivariate Bayesian Logistic Regression*** analysis


In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pymc as pm

In [ ]:
# Loading data
# Dataset: https://www.kaggle.com/datasets/nelgiriyewithana/apple-quality
# Note: Authors name was manually removed from the bottom of the csv file to
# remove `Quality` column conversion process.
data = pd.read_csv("apple_quality.csv")

# Binarize the 'Quality' column where 'good' = 1, 'bad' = 0
data['Quality'] = data['Quality'].apply(lambda x: 1 if x == 'good' else 0)

data.head()

,A_id,Size,Weight,Sweetness,Crunchiness,Juiciness,Ripeness,Acidity,Quality
0,0,-3.970049,-2.512336,5.346330,-1.012009,1.844900,0.329840,-0.491590,1
1,1,-1.195217,-2.839257,3.664059,1.588232,0.853286,0.867530,-0.722809,1
2,2,-0.292024,-1.351282,-1.738429,-0.342616,2.838636,-0.038033,2.621636,0
3,3,-0.657196,-2.271627,1.324874,-0.097875,3.637970,-3.413761,0.790723,1
4,4,1.364217,-1.296612,-0.384658,-0.553006,3.030874,-1.303849,0.501984,1


To do a univariate bayesian logistic regression, the outcome column `Quality` will use the predictor `Sweetness` to develop this model. Moreover, the starter code is adjusted to have a Bernoulli distribution using a logit-link function.



In [ ]:
# Data selection
X = data['Juiciness'].values
y = data['Quality'].values

In [ ]:
# Univariate Bayesian Logistic Regression (using logit-link function)
with pm.Model() as UBLR:

    intercept = pm.Normal('Intercept', mu=0, sigma=1)
    slope = pm.Normal('Slope', mu=0, sigma=1)

    linear_combination = intercept + slope * X

    p = pm.math.invlogit(linear_combination)

    quality_obs = pm.Bernoulli('Quality_obs', p, observed=y)

with UBLR:
    idata = pm.sample()

## Part II<br>Regularized Loss Functions

***Machine Learning*** fits models by optimizing penalized ***loss functions***

Two classic regularizations are "ridge" and "lasso" regression, which respectively use $L_1$ and $L_2$ penalty functions

- Ridge: $$\sum_{i=1}^n \frac{1}{2}(y-x_i^T\beta_{p \times 1})^2 + \lambda \sum_{j=1}^n |\beta_j| = \frac{1}{2}(y-X\beta)^T(y-X\beta) + \lambda \sum_{j=1}^n |\beta_j| = \frac{1}{2}||y-X\beta||_2^2 + \lambda ||\beta||_1$$
- Lasso: $$\sum_{i=1}^n \frac{1}{2}(y_i-x_i^T\beta_{p \times 1})^2 + \lambda \sum_{j=1}^n \beta_j^2 = \frac{1}{2}(y-X\beta)^T(y-X\beta) + \lambda \sum_{j=1}^n \beta_j^2 = \frac{1}{2}||y-X\beta||_2^2 + \lambda ||\beta||_2^2 $$
    
Show that for $\sigma=1$ and ***hyperparameters*** $b_i=0$ (ignoring normalizing proportionality constants) the log posterior distributions for $\beta$ using either ***normal*** or ***Laplace*** prior distributions have analagous forms to the above expressions

Now write down and understand the following: "Bayesians do not optimize posterior distributions, they sample from them; but, the posterior distributions are nonetheless 'regularizations' of the likelihood through the prior."

### Solution
Let $\sigma = 1$ and $b_{i} = 0$. In the text cell titled ["Bayesian Shrinkage Estimation: "Lasso Regression"](https://github.com/pointOfive/STA365_W24_Bayes/blob/main/Bayes6_FancyRegression-Clickable.ipynb), we are given the following priors (ignoring normalizing proprtionality constants and using $\sigma = 1$ and $b_{i} = 0$):

- Lasso Regression
\begin{align*}
 \beta_i \sim {} & \text{Normal}(0,s_i) & y_i \sim {} & \text{Normal}(x_i^T\beta, 1) & f(\beta_i| 0, s_i) = {}& {\frac  {1}{2s_{i}}}\exp \left(-{\frac{|\beta_i|}{s_i}}\right)
\end{align*}
- Ridge Regression
\begin{align*}
\beta_i \sim {} & \text{Laplace}(0,s_i) & y_i \sim {} & \text{Normal}(x_i^T\beta, 1) & f(\beta_i| 0, s_i) = {}& \frac{1}{\sqrt{2\pi}}e^{-\frac{1}{2}\left(\frac{\beta_i}{s_i}\right)^2}
\end{align*}

Assuming we ignore the normalizing constants, we can proceed by taking the log posterior as follows for both regression models, where $\mathbf{y}$ and $\mathbf{x}$ being vectors of dependent and independent variables respectively.

#### Ridge Regression

\begin{align*}
\log\left(f(\beta_i | \mathbf{y}, \mathbf{X})\right) &= \log\left(\prod^{n}_{i=1} f(y_{i} | x_{i}, \beta, \sigma=1) \prod^{n}_{j=1} f(\beta_{j} | 0, s_{j})\right) \\
&= \sum^{n}_{i=1}\log\left(f(y_{i}|x_{i}, \beta, \sigma=1)\right) + \sum^{n}_{j=1}\log\left(f(\beta_{j}|0, s_{j})\right) &\text{(by log properties)}\\
&= -\frac{1}{2}\sum^{n}_{i=1}(y_{i} - x_{i}^{T}\beta)^{2} - \frac{1}{2}\sum^{n}_{j=1} \left(\frac{\beta_{j}}{s_{j}}\right)^{2} &\text{(by log properties)}\\
&= -\frac{1}{2}\sum^{n}_{i=1}(y_{i} - x_{i}^{T}\beta)^{2} - \frac{1}{2}\sum^{n}_{j=1} \frac{\beta_{j}^{2}}{s_{j}^{2}}
\end{align*}

Taking $\lambda = \frac{1}{2s_{j}^{2}}$,

\begin{align*}
-\frac{1}{2}\sum^{n}_{i=1}(y_{i} - x_{i}^{T}\beta)^{2} - \frac{1}{2}\sum^{n}_{j=1} \frac{\beta_{j}^{2}}{s_{j}^{2}} &= -\frac{1}{2}\sum^{n}_{i=1}(y_{i} - x_{i}^{T}\beta)^{2} - \lambda\sum^{n}_{j=1}\beta_{j}^{2}
\end{align*}

Since the objective here is to optimize penalized loss functions, the negative signs can be changed to positives as we can notice that the loss functions are convex and correspond to some minimization. Therefore,

\begin{align*}
\frac{1}{2}\sum^{n}_{i=1}(y_{i} - x_{i}^{T}\beta)^{2} + \lambda\sum^{n}_{j=1}\beta_{j}^{2}
\end{align*}

#### Lasso Regression
\begin{align*}
\log\left(f(\beta_i | \mathbf{y}, \mathbf{X})\right) &= \log\left(\prod^{n}_{i=1} f(y_{i} | x_{i}, \beta, \sigma=1) \prod^{n}_{j=1} f(\beta_{j} | 0, s_{j})\right) \\
&= \sum^{n}_{i=1}\log\left(f(y_{i}|x_{i}, \beta, \sigma=1)\right) + \sum^{n}_{j=1}\log\left(f(\beta_{j}|0, s_{j})\right) &\text{(by log properties)}\\
&= -\frac{1}{2}\sum^{n}_{i=1}(y_{i} - x_{i}^{T}\beta)^{2} - \frac{1}{2}\sum^{n}_{j=1} \frac{|\beta_{j}|}{s_{j}} &\text{(by log properties)}\\
\end{align*}

Taking $\lambda = \frac{1}{s_{j}}$,

\begin{align*}
-\frac{1}{2}\sum^{n}_{i=1}(y_{i} - x_{i}^{T}\beta)^{2} - \frac{1}{2}\sum^{n}_{j=1} \frac{|\beta_{j}|}{s_{j}} &= -\frac{1}{2}\sum^{n}_{i=1}(y_{i} - x_{i}^{T}\beta)^{2} - \lambda\sum^{n}_{j=1} |\beta_{j}|
\end{align*}

Using the same logic from earlier to justify changing the negative signs to positive,

\begin{align*}
\frac{1}{2}\sum^{n}_{i=1}(y_{i} - x_{i}^{T}\beta)^{2} + \lambda\sum^{n}_{j=1} |\beta_{j}|
\end{align*}


#### Statement
"Bayesians do not optimize posterior distributions, they sample from them; but, the posterior distributions are nonetheless 'regularizations' of the likelihood through the prior."